In [1]:
import os
import zipfile
import pandas as pd
from tqdm import tqdm
# import warnings
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

In [2]:
demo_outfile = '../data/demographic.csv'
hist_outfile = '../data/vote_history.csv'

In [ ]:
demo_flag = True
hist_flag = True

for subdir, dirs, files in os.walk("/Users/matt/StateVoterFiles/"):
    for file in tqdm(files):
        if file.endswith(".zip"):
            archive_name = os.path.join(subdir, file)
            
            state_abbrv = file[5:7]
            print(state_abbrv)
                        
            with zipfile.ZipFile(archive_name, 'r') as archive:
#                 try:
                for subfile in archive.namelist():
                    if subfile.endswith("DEMOGRAPHIC.tab"):
                        demo = pd.read_csv(
                            archive.open(subfile),
                            sep='\t',
                            usecols = ['LALVOTERID', 'Voters_Age', 'County'],
                            chunksize = 10000
                        )

                        for chunk in demo:
                            foo = chunk.copy()
                            foo['State'] = state_abbrv
                            if demo_flag:
                                foo.to_csv(demo_outfile, index = False)
                                demo_flag = False
                            else:
                                foo.to_csv(demo_outfile, mode='a', header=False, index = False)

                    elif subfile.endswith("VOTEHISTORY.tab"):
                        hist = pd.read_csv(
                            archive.open(subfile),
                            sep='\t',
                            usecols = ['LALVOTERID', 'General_2020_11_03'],
                            chunksize = 10000
                        )

                        for chunk in hist:
                            foo = chunk.copy()
                            foo['State'] = state_abbrv
                            if hist_flag:
                                foo.to_csv(hist_outfile, index = False)
                                hist_flag = False
                            else:
                                foo.to_csv(hist_outfile, mode='a', header=False, index = False)     

#                 except:
#                     print('FAILURE: ' + file)

In [34]:
# I should have stored each state's files separately from the beginning,
# but I didn't think of that until it was too late

demo = pd.read_csv('../data/demographic.csv', chunksize = 100000)

state_map = {}
for chunk in tqdm(demo):
    states = chunk['State'].unique()
    for state in states:
        if state not in state_map:
            state_map[state] = '../data/vf_demographic/' + state + '.csv'
            chunk[chunk.State == state].to_csv(state_map[state], index = False)
        else:
            chunk[chunk.State == state].to_csv(state_map[state], mode='a', header=False, index = False)

142it [00:51,  2.75it/s]


In [35]:
hist = pd.read_csv('../data/vote_history.csv', chunksize = 100000)

state_map = {}
for chunk in tqdm(hist):
    states = chunk['State'].unique()
    for state in states:
        if state not in state_map:
            state_map[state] = '../data/vf_history/' + state + '.csv'
            chunk[chunk.State == state].to_csv(state_map[state], index = False)
        else:
            chunk[chunk.State == state].to_csv(state_map[state], mode='a', header=False, index = False)

142it [00:31,  4.52it/s]


In [36]:
import os

os.remove('../data/demographic.csv')
os.remove('../data/vote_history.csv')